# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [28]:
df = data.describe(include='object')

df


,Education,Occupation,Commute Distance,Region
count,1000,1000,1000,1000
unique,5,5,5,3
top,Bachelors,Professional,0-1 Miles,North America
freq,306,276,366,508


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [26]:
for column in data.columns:
    unique_values = data[column].unique()

    # Проверяем, что в колонке только два уникальных значения
    if len(unique_values) == 2:
        
        mode_values = data[column].mode()
        
        if len(mode_values) == 1:
            # если мода одна
            mode_value = mode_values.iloc[0]
            other_value = unique_values[unique_values != mode_value][0]
            mode_map = {mode_value: 1, other_value: 0}
        elif len(mode_values) == 2:
            # если есть две моды (редкий случай)
            mode_map = {mode_values.iloc[0]: 1, mode_values.iloc[1]: 0}
        
        data[column] = data[column].map(mode_map)

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [32]:
columns_to_drop = [column for column in data.columns if column in df.columns] # 4
data = data.drop(columns_to_drop, axis=1)
data

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,1,2,54,0
996,28672,0,1,70000,4,1,0,35,0
997,11809,1,1,60000,2,1,0,38,0
998,19664,0,1,100000,3,0,3,38,1


Удалите столбец `ID`, так как он по сути нечисловой.

In [35]:

data

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,1,0,40000,1,1,0,42,1
1,1,1,30000,3,1,1,43,1
2,1,1,80000,5,0,2,60,1
3,0,1,70000,0,1,1,41,0
4,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...
995,1,1,60000,2,1,2,54,0
996,0,1,70000,4,1,0,35,0
997,1,1,60000,2,1,0,38,0
998,0,1,100000,3,0,3,38,1


Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [36]:
X = data.drop('Purchased Bike', axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [49]:
X_train.shape[1] #700

7

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [50]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Теперь обучите логистическую регрессию на тренировочных данных

In [41]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [42]:
predict_train = model.predict(X_train)
predict_test = model.predict(X_test)

Оцените значение accuracy на трейне и на тесте.

In [44]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train, predict_train)
accuracy_test = accuracy_score(y_test, predict_test)
print(accuracy_train, accuracy_test)

0.6342857142857142 0.58


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [51]:
from sklearn.preprocessing import PolynomialFeatures
features = PolynomialFeatures(degree=2)
X_train = features.fit_transform(X_train)
X_test = features.transform(X_test)
X_train.shape

(700, 36)

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [53]:
#29 
model = LogisticRegression()
model.fit(X_train, y_train)
predict_train = model.predict(X_train)
predict_test = model.predict(X_test)
accuracy_train = accuracy_score(y_train, predict_train)
accuracy_test = accuracy_score(y_test, predict_test)
print(accuracy_train, accuracy_test)

0.6871428571428572 0.6233333333333333


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [55]:
from sklearn.metrics import recall_score
rc = recall_score(y_test, predict_test)
rc

0.6959459459459459

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [57]:
XtrainS, Xval, ytrainS, yval = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [82]:
model.fit(XtrainS, ytrainS)
predict = model.predict_proba(Xval)[:,1]

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    predict1 = (predict >= thr).astype(int)
    current_recall = recall_score(yval, predict1)
    current_accuracy = accuracy_score(yval, predict1)
    
    if current_recall > RecMax and current_accuracy >= 0.6:
        RecMax = current_recall
        BestAcc = current_accuracy
        BestThr = thr

print(BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [80]:
model.fit(X_train, y_train)
predict = model.predict_proba(X_test)[:,1]
predict = (predict >= BestThr).astype(int)
recall_score(y_test, predict)

0.8040540540540541

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.